In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from regressor.equation_utils import pretty_print_guess_tensor, eqn_to_block_tensor
from regressor.train import bind_opt_train_step

In [3]:
fn = lambda g, m1, m2, r, one, zero: ((g * m1) * (zero + m2)) / ((r * r) + (one * zero)) # (G * m1 * m2) / (r * r)
# fn = lambda g, m1, m2, r, one, zero: ((m1 + m2) + (zero + zero)) + ((zero + zero) + (zero + zero))

G = 1 # 6.67408e-11
ONE = 1
ZERO = 0

fn(G, 2, 2, 2, ONE, ZERO)

1.0

In [4]:
NUM_DATAPOINTS = 2
NUM_VARS = 6
LOW = 1
HIGH = 10

xs = np.reshape(np.random.uniform(LOW, HIGH, [NUM_DATAPOINTS * NUM_VARS]), [NUM_DATAPOINTS, NUM_VARS])
xs[:,0] = G
xs[:,4] = ONE
xs[:,5] = ZERO
xs

array([[1.        , 1.66251499, 3.31238004, 9.96540146, 1.        ,
        0.        ],
       [1.        , 4.25640232, 2.52140422, 6.57827311, 1.        ,
        0.        ]])

In [5]:
eqn_to_block_tensor(fn, xs)

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>,
 <tf.Tensor: shape=(1, 8, 6), dtype=float32, numpy=
 array([[[1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 3, 4, 4), dtype=float32, numpy=
 array([[[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]],
 
         [[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]],
 
         [[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 2, 6), dtype=float32, numpy=
 array([[[1.       , 1.662515 , 3.31238  , 9.965402 , 1.       ,
          0.       ],
         [1.      

In [6]:
NUM_DATAPOINTS = 100
NUM_VARS = 6
LOW = 1e-1
HIGH = 1e+1
TREE_DEPTH = 3

xs = np.reshape(np.random.uniform(LOW, HIGH, [NUM_DATAPOINTS * NUM_VARS]), [NUM_DATAPOINTS, NUM_VARS])
# xs = np.array([[1, 20,30,40, 1,0]], dtype=np.float32)

xs[:,0] = G
xs[:,4] = ONE
xs[:,5] = ZERO

In [7]:
constant_guess, operand_guess, operator_guess, values, target = eqn_to_block_tensor(fn, xs, TREE_DEPTH)
# tf.print('constant_guess', tf.shape(constant_guess)) # TODO
tf.print('operand_guess', tf.shape(operand_guess))
tf.print('operator_guess', tf.shape(operator_guess))
tf.print('values', tf.shape(values))
tf.print('target', tf.shape(target))

constant_guess = tf.Variable(constant_guess) # TODO
operand_guess = tf.Variable(operand_guess)
operator_guess = tf.Variable(operator_guess)

operand_guess [1 8 6]
operator_guess [1 3 4 4]
values [1 100 6]
target [1 100 1]


In [8]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-1,
    decay_steps=100,
    decay_rate=1e-1,
    staircase=True)
opt = tf.keras.optimizers.Adam(lr_schedule)
train_step = bind_opt_train_step(opt, TREE_DEPTH)

In [9]:
steps = 1000
for i in range(steps):
    loss, result, grads = train_step(constant_guess, operand_guess, operator_guess, values, target)

    if i % (steps // 10) == 0:
        print_row = lambda x: pretty_print_guess_tensor(constant_guess[x], operand_guess[x], operator_guess[x])
        eqn_str = print_row(0).replace('x_0', 'G').replace('x_1', 'm1').replace('x_2', 'm2').replace('x_3', 'r').replace('x_4', '1').replace('x_5', '0')
        tf.print(loss,eqn_str)
print(tf.round(grads[0]))
print(tf.round(grads[1]))

201879.469 (((G + m1) + (m2 - r)) + ((1 * 0) - (G / m1)))
207869.813 (((0 + 0) + (0 + 0)) - ((0 + 0) - (m1 + 0)))
202304.766 (((0 + 0) + (0 + 0)) - ((0 + 0) - (m1 + m1)))
202304.766 (((0 + 0) + (0 + 0)) - ((0 + 0) - (m1 + m1)))
202304.766 (((0 + 0) + (0 + 0)) - ((0 + 0) - (m1 + m1)))
202304.766 (((0 + 0) + (0 + 0)) - ((0 + 0) - (m1 + m1)))
202304.766 (((0 + 0) + (0 + 0)) - ((0 + 0) - (m1 + m1)))
202304.766 (((0 + 0) + (0 + 0)) - ((0 + 0) - (m1 + m1)))
202304.766 (((0 + 0) + (0 + 0)) - ((0 + 0) - (m1 + m1)))
202304.766 (((0 + 0) + (0 + 0)) - ((0 + 0) - (m1 + m1)))
tf.Tensor(
[[[ 1722.  -169.  1645.  1709.  1722.   258.]
  [ 1720.  -168.  1643.  1707.  1720.   258.]
  [ 1720.  -168.  1643.  1707.  1720.   258.]
  [ 1717.  -168.  1641.  1704.  1717.   257.]
  [ 1545.   895.  1304.  1167.  1558.  -238.]
  [ 1542.   893.  1302.  1165.  1555.  -238.]
  [ 7209. -1717.  7004.  7204.  7209.  1928.]
  [ 7206. -1717.  7001.  7201.  7206.  1927.]]], shape=(1, 8, 6), dtype=float32)
tf.Tensor(
[[[[-

In [10]:
tf.round(operand_guess * 100)

<tf.Tensor: shape=(1, 8, 6), dtype=float32, numpy=
array([[[  0.,   1.,   0.,   0.,   0.,  99.],
        [  0.,   1.,   0.,   0.,   0.,  99.],
        [  0.,   1.,   0.,   0.,   0.,  99.],
        [  0.,   0.,   0.,   0.,   0.,  99.],
        [  0.,   0.,   0.,   0.,   0., 100.],
        [  0.,   0.,   0.,   0.,   0., 100.],
        [  0.,  70.,   0.,   0.,   0.,  29.],
        [  0.,  72.,   0.,   0.,   0.,  28.]]], dtype=float32)>

In [11]:
tf.round(operator_guess * 100)

<tf.Tensor: shape=(1, 3, 4, 4), dtype=float32, numpy=
array([[[[100.,   0.,   0.,   0.],
         [100.,   0.,   0.,   0.],
         [ 88.,   0.,  12.,   0.],
         [ 54.,   0.,  46.,   0.]],

        [[100.,   0.,   0.,   0.],
         [  0., 100.,   0.,   0.],
         [  0.,   0., 100.,   0.],
         [  0.,   0.,   0., 100.]],

        [[  0., 100.,   0.,   0.],
         [  0., 100.,   0.,   0.],
         [  0.,   0., 100.,   0.],
         [  0.,   0.,   0., 100.]]]], dtype=float32)>